# Basics

Import libraries and functions.
.

</p>Search for archives, and create the 1st data frame, as well as an empty list for the headers (h) and the list for comun columns (CC)

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from pyspark.sql.functions import concat, col, lit

<p>Using the <code>glob</code> library allows us to create a list with only the csv that are ended in <em>(Normalized).csv</em> , which will be the most useful for a statistical analysis. <br>
Moreover, for a future simplification, we also create a list with the relevant information of each of the csv <em>('Area','Year','Element','Unit','Value')</em> </p>

In [6]:
file_list = glob.glob(os.getcwd()+ "/Data/"+"/**(Normalized).csv")
CC=['Area','Year','Element' or 'Item','Unit','Value']

Intitiate the spark session

In [7]:
from pyspark.sql import SparkSession
  
ss = SparkSession \
    .builder \
    .appName("Our First Spark example") \
    .getOrCreate()

Header locator, to know which are the variables in each csv. 

In [39]:
for i in range (0,len(file_list)):
    file1=ss.read.csv(file_list[i], header=True)
    if 'Element' in file1.schema.fieldNames():
        if 'Item' in file1.schema.fieldNames():
            if 'Area' in file1.schema.fieldNames():
                if 'Year' in file1.schema.fieldNames():
                    if 'Unit' in file1.schema.fieldNames():
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
                    else:
                        print(i)
                        print('Unit')
                        print(file1.schema.fieldNames())    
                else:
                    print(i)
                    print('Year')
                    print(file1.schema.fieldNames())
            else:
                print(i)
                print('Area')
                print(file1.schema.fieldNames())
        else:
            print(i)
            print('Item')
            print(file1.schema.fieldNames())
    else:
        print(i) 
        print('Element')
        print(file1.schema.fieldNames())
        if 'Item' in file1.schema.fieldNames():
            if 'Area' in file1.schema.fieldNames():
                if 'Year' in file1.schema.fieldNames():
                    if 'Unit' in file1.schema.fieldNames():
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
                    else:
                        print(i)
                        print('Unit')
                        print(file1.schema.fieldNames())    
                else:
                    print(i)
                    print('Year')
                    print(file1.schema.fieldNames())
            else:
                print(i)
                print('Area')
                print(file1.schema.fieldNames())
        else:
            print(i)
            print('Item')
            print(file1.schema.fieldNames())
            if 'Area' in file1.schema.fieldNames():
                if 'Year' in file1.schema.fieldNames():
                    if 'Unit' in file1.schema.fieldNames():
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
                    else:
                        print(i)
                        print('Unit')
                        print(file1.schema.fieldNames())    
                else:
                    print(i)
                    print('Year')
                    print(file1.schema.fieldNames())
            else:
                print(i)
                print('Area')
                print(file1.schema.fieldNames())
                if 'Year' in file1.schema.fieldNames():
                    if 'Unit' in file1.schema.fieldNames():
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
                    else:
                        print(i)
                        print('Unit')
                        print(file1.schema.fieldNames())    
                else:
                    print(i)
                    print('Year')
                    print(file1.schema.fieldNames())
                    if 'Unit' in file1.schema.fieldNames():
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
                    else:
                        print(i)
                        print('Unit')
                        print(file1.schema.fieldNames())
                        if 'Value' in file1.schema.fieldNames():
                            pass
                        else:
                            print(i)
                            print('Value')
                            print(file1.schema.fieldNames())
            

3
Element
['Area Code', 'Area', 'Item Code', 'Item', 'Months Code', 'Months', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
5
Area
['Donor Code', 'Donor', 'Recipient Country Code', 'Recipient Country', 'Item Code', 'Item', 'Element Code', 'Element', 'Purpose Code', 'Purpose', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
20
Element
['Area Code', 'Area', 'Source Code', 'Source', 'Indicator Code', 'Indicator', 'Sex Code', 'Sex', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
20
Item
['Area Code', 'Area', 'Source Code', 'Source', 'Indicator Code', 'Indicator', 'Sex Code', 'Sex', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
21
Element
['Area Code', 'Area', 'Source Code', 'Source', 'Indicator Code', 'Indicator', 'Sex Code', 'Sex', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
21
Item
['Area Code', 'Area', 'Source Code', 'Source', 'Indicator Code', 'Indicator', 'Sex Code', 'Sex', 'Year Code', 'Year', 'Unit', 'Value', 'Flag', 'Note']
22
Element
['Area C

The following cell will allow to read  the csv files from the main directory

In [8]:
file0 = ss.read.csv(file_list[0], sep=',',header= True)
file0=file0.select('Area','Year', (concat(col('Item'),lit(' - '), col('Element'),lit(' - '),col('Unit'))).alias('Unique'),'Value')
file0.show(truncate=False)

+-------------------+----+--------------------------------------------------------------------------------------------+---------+
|Area               |Year|Unique                                                                                      |Value    |
+-------------------+----+--------------------------------------------------------------------------------------------+---------+
|Algeria            |2009|Agriculture research spending - Share of Value Added (Agriculture, Forestry and Fishing) - %|0.180000 |
|Algeria            |2010|Agriculture research spending - Share of Value Added (Agriculture, Forestry and Fishing) - %|0.180000 |
|Algeria            |2011|Agriculture research spending - Share of Value Added (Agriculture, Forestry and Fishing) - %|0.210000 |
|Algeria            |2012|Agriculture research spending - Share of Value Added (Agriculture, Forestry and Fishing) - %|0.210000 |
|Algeria            |2009|Agriculture research spending - Spending, total - million PPP (c

Loop attempt; not everyone has element or item

In [37]:
for i in range (1,len(file_list)):
    file1=ss.read.csv(file_list[i], header=True)
    file1=file1.select('Area','Year', (concat(col('Item'),lit(' - '), col('Element'),lit(' - '),col('Unit'))).alias('Unique'),'Value')
    file0=file0.union(file1)

AnalysisException: cannot resolve 'Element' given input columns: [Area, Area Code, Flag, Item, Item Code, Months, Months Code, Note, Unit, Value, Year, Year Code];
'Project [Area#33975, Year#33981, concat(Item#33977,  - , 'Element,  - , Unit#33982) AS Unique#33998, Value#33983]
+- Relation [Area Code#33974,Area#33975,Item Code#33976,Item#33977,Months Code#33978,Months#33979,Year Code#33980,Year#33981,Unit#33982,Value#33983,Flag#33984,Note#33985] csv


 Loop attempt2; looks promising, not finished.

In [12]:
for i in range (1,len(file_list)):
    if i == # put number where the damm archive is:
        continue
    else:
        file1=ss.read.csv(file_list[i], header=True)
        if 'Element' in file1.schema.fieldNames():
            if 'Item' in file1.schema.fieldNames():
                if 'Area' in file1.schema.fieldNames():
                    pass 
                else:
                    file1=file1.withColumn('Area',col('Recipient Country'))
            else:
                file1=file1.withColumn('Item', lit(0))
        else:
            file1=file1.withColumn('Element', lit(0))
            if 'Item' in file1.schema.fieldNames():
                pass
            else:
                file1=file1.withColumn('Item', lit(0))
        file1=file1.select('Area' or 'Country','Year', (concat(col('Item'),lit(' - '), col('Element'),lit(' - '),col('Unit'))).alias('Unique'),'Value')
        file0=file0.union(file1)

SyntaxError: invalid syntax (1332274217.py, line 2)

In [59]:
file1=ss.read.csv(file_list[18], header=True)
[element for element in file1.columns if 'Cod' in element]


['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Source Code']

# Data integration through pandas

As at the beginning we do not have enough computer power to process the whole database, we are going to develop a test run to be sure that the ideas are escalable.


In this first test run for the loop, we are going to load and process the data into concated data frames, where later on, there is an application of the <code>pivot_table</code> function which adjusts all the variables, previouly called 'Elements' & 'Units' into the headers of the columns, and the values the values of the table.

In [ ]:
main_dataframe = pd.DataFrame(pd.read_csv(file_list[0], sep=',', encoding='latin-1'),columns=CC)
for i in range(1,5):
    df = pd.DataFrame(pd.read_csv(file_list[i],sep=',' , encoding='latin-1',low_memory=False), columns=CC)
    main_dataframe = pd.concat([main_dataframe, df])

main_dataframeC=main_dataframe.pivot_table(index=['Area','Year'], columns= ['Element' or 'Item','Unit'], values='Value')
main_dataframeC

<p>In the following cell, we are concatanating all the files from the <code>file_list</code>, which will have the same shape thanks to creation of the dataframes with the restriction of the columns. <br>
Moreover, this concat function will allow for a single data frame which has all the files one on top of another. Therefore the final result form this loop will be <code>main_dataframe</code> which will be our Normalized Source Data Model.</p>

In [ ]:
main_dataframe = pd.DataFrame(pd.read_csv(file_list[0], sep=',', encoding='latin-1'),columns=CC)
for i in range(1,len(file_list)):
    df = pd.DataFrame(pd.read_csv(file_list[i],sep=',' , encoding='latin-1',low_memory=False), columns=CC)
    main_dataframe = pd.concat([main_dataframe, df])

<p>Lastly, to convert the Normalized Source Data Model into the Normalized Integrated Data Model, we are going to use the <code>pivot_table</code> function which allows to <br>
adjusts all the variables, previouly called <em>'Elements' & 'Units'</em> into the headers of the columns, and the <em>'Value'</em> column will be the values of the table.

In [ ]:
main_dataframeC=main_dataframe.pivot_table(index=['Area','Year'], columns= ['Element' or 'Item' ,'Unit'], values='Value')
main_dataframeC

# Quality assurance

In the following cell, we are going to make sure that none of our interesting variables from <em>'Elements'</em> have been left out, thus checking if the extraction & integtration has been completed.

In [12]:
if len(main_dataframe["Element" or 'Item'].value_counts())==main_dataframeC.shape[1]:
    print('Data extraction & integration is COMPLETED and CORRECT')
else:
    print('Data extraction & integration is UNCOMPLETED')

Data extraction & integration is COMPLETED and CORRECT
